In [ ]:
import numpy as np 
import pandas as pd 
import cv2

import gc

from glob import glob
import os

from vtk import vtkJPEGReader 
from vtk.util import numpy_support

from joblib import Parallel, delayed
from tqdm import tqdm_notebook

import matplotlib.pyplot as plt

import tensorflow.keras.layers as L
from tensorflow.keras.models import Model
import tensorflow.keras.optimizers as OPTS

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [ ]:
%time
train = pd.read_csv('/kaggle/input/shopee-product-matching/train.csv')

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
train['label_group'].value_counts(),len(train['label_group'].value_counts())

In [ ]:
train['image_phash'].apply(len).value_counts()

In [ ]:
train_paths = glob('/kaggle/input/shopee-product-matching/train_images/*')

In [ ]:
reader = vtkJPEGReader()
#ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, 3]
# ConstPixelDims = [224,224,3]
def get_shape(path):
    reader.SetFileName(path)
    reader.Update()
    _extent = reader.GetDataExtent()
    ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, 3]
    print(ConstPixelDims)

In [ ]:
for i in range(5):
    get_shape(train_paths[i])

In [ ]:
reader = vtkJPEGReader()
#ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, 3]
# ConstPixelDims = [224,224,3]
def get_img(path):
    reader.SetFileName(path)
    reader.Update()
    _extent = reader.GetDataExtent()
    ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, 3]
    imageData = reader.GetOutput()
    pointData = imageData.GetPointData()
    arrayData = pointData.GetArray(0)
    im = numpy_support.vtk_to_numpy(arrayData)
    im = im.reshape(ConstPixelDims, order='F')
    return im

In [ ]:
for i in range(5):
    plt.imshow(get_img(train_paths[i]))
    plt.show()

In [ ]:
target = train['label_group']
train.drop(['posting_id','image','label_group'],axis=1,inplace = True)

In [ ]:
hash_characters = train['image_phash'].apply(lambda x : list(x))

In [ ]:
hash_characters = pd.DataFrame(hash_characters.explode().values.flatten().reshape(-1,16))

In [ ]:
character_map = dict(zip(np.unique(hash_characters),np.arange(len(np.unique(hash_characters)))))

In [ ]:
character_map

In [ ]:
hash_characters = hash_characters.replace(character_map)

In [ ]:
hash_characters

In [ ]:
clf = XGBClassifier(tree_method = 'gpu_hist')

In [ ]:
clf.fit(hash_characters,target)

In [ ]:
target_pred = clf.predict(hash_characters)

In [ ]:
(target_pred == target).sum()/len(target)

In [ ]:
# /kaggle/input/shopee-product-matching/sample_submission.csv
# /kaggle/input/shopee-product-matching/train.csv
# /kaggle/input/shopee-product-matching/test.csv